In [ ]:

# import os
# import sys
# from tempfile import NamedTemporaryFile
# from urllib.request import urlopen
# from urllib.parse import unquote, urlparse
# from urllib.error import HTTPError
# from zipfile import ZipFile
# import tarfile
# import shutil

# CHUNK_SIZE = 40960
# DATA_SOURCE_MAPPING = 'pascalvoc-yolo:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F918769%2F1556326%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240803%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240803T180619Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D60245d936b12665b478461fae2417841ba2c7203275a0ae79659cb349615d122fac7b15d780d8891bb36e299c6bdf680fe6859446ca67e70d8d5432d2125cdeaf545729e8416e98cee7d35e9b170ab7f6cc74a90bfe13132a89fb9d1e010d24a968018b5af2820483069c50f9e19a704c76552339d4d6b9e706cd97f54082ed42e35db0153b33745f56f4ba8add1879563830b759076dc443bdaf5059a6d9b14ac3fffea278ebe72b2599b5f1aa4ac69e61b10489dd613fa15191ff298552939ad0b28066478f9d2aea9cb4559e8b32b431070c89b1d3ea223d49a482bffa1dc5916f86a5a8d4457e1b12276d4f19e028cfbc32bff990ed382347814dff79301'

# KAGGLE_INPUT_PATH='/kaggle/input'
# KAGGLE_WORKING_PATH='/kaggle/working'
# KAGGLE_SYMLINK='kaggle'

# !umount /kaggle/input/ 2> /dev/null
# shutil.rmtree('/kaggle/input', ignore_errors=True)
# os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
# os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

# try:
#   os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
# except FileExistsError:
#   pass
# try:
#   os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
# except FileExistsError:
#   pass

# for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
#     directory, download_url_encoded = data_source_mapping.split(':')
#     download_url = unquote(download_url_encoded)
#     filename = urlparse(download_url).path
#     destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
#     try:
#         with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
#             total_length = fileres.headers['content-length']
#             print(f'Downloading {directory}, {total_length} bytes compressed')
#             dl = 0
#             data = fileres.read(CHUNK_SIZE)
#             while len(data) > 0:
#                 dl += len(data)
#                 tfile.write(data)
#                 done = int(50 * dl / int(total_length))
#                 sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
#                 sys.stdout.flush()
#                 data = fileres.read(CHUNK_SIZE)
#             if filename.endswith('.zip'):
#               with ZipFile(tfile) as zfile:
#                 zfile.extractall(destination_path)
#             else:
#               with tarfile.open(tfile.name) as tarfile:
#                 tarfile.extractall(destination_path)
#             print(f'\nDownloaded and uncompressed: {directory}')
#     except HTTPError as e:
#         print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
#         continue
#     except OSError as e:
#         print(f'Failed to load {download_url} to path {destination_path}')
#         continue

# print('Data source import complete.')


In [ ]:
# !pip install wandb

In [ ]:
import os
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image, ImageFile
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import wandb  # Import wandb
from transformers import BertTokenizer

In [ ]:
# Login with the API KEY
wandb.login(key="ab35ea8191eba471c2b58a844910531625b00550")

# Initialize wandb
wandb.init(project="Untitled10", entity="mblogge785-work")  # Replace with your wandb username

In [ ]:
# Initialize the tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [ ]:
# Define configuration for YOLOv3
config = [
    (32, 3, 1),
    (128, 3, 1),
    (64, 3, 2),
    ["list", 1],
    (128, 3, 2),
    ["list", 2],
    (256, 3, 2),
    ["list", 8],
    (512, 3, 2),
    ["list", 8],
    (1024, 3, 2),
    ["list", 4],
    (512, 1, 1),
    (1024, 3, 1),
    "sp",
    (256, 1, 1),
    "up",
    (256, 1, 1),
    (512, 3, 1),
    "sp",
    (128, 1, 1),
    "up",
    (128, 1, 1),
    (256, 3, 1),
    "sp",
]

In [ ]:
# Define CNN block
class CNN_Block(nn.Module):
    def __init__(self, in_channels, out_channels, bn_act=True, **kwargs):
        super(CNN_Block, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, bias=not bn_act, **kwargs)
        self.bn = nn.BatchNorm2d(out_channels)
        self.leaky = nn.LeakyReLU(0.1)
        self.use_bn_act = bn_act

    def forward(self, x):
        if self.use_bn_act:
            return self.leaky(self.bn(self.conv(x)))
        else:
            return self.conv(x)

In [ ]:
# Define Residual block
class Residual_Block(nn.Module):
    def __init__(self, channels, use_residual=True, num_repeats=1):
        super(Residual_Block, self).__init__()
        self.layers = nn.ModuleList()

        for repeat in range(num_repeats):
            self.layers += [
                nn.Sequential(
                    CNN_Block(channels, channels//2, kernel_size=1),
                    CNN_Block(channels//2, channels, kernel_size=3, padding=1)
                )
            ]

        self.use_residual = use_residual
        self.num_repeats = num_repeats

    def forward(self, x):
        for layer in self.layers:
            if self.use_residual:
                x = x + layer(x)
            else:
                x = layer(x)

        return x

In [ ]:
# Define Prediction Scale
class Prediction_Scale(nn.Module):
    def __init__(self, in_channels, NumClasses):
        super(Prediction_Scale, self).__init__()
        self.pred = nn.Sequential(
            CNN_Block(in_channels, 2 * in_channels, kernel_size=3, padding=1),
            CNN_Block(2 * in_channels, (NumClasses + 5) * 3, bn_act=False, kernel_size=1),
        )
        self.NumClasses = NumClasses

    def forward(self, x):
        return (
            self.pred(x)
            .reshape(x.shape[0], 3, self.NumClasses + 5, x.shape[2], x.shape[3])
            .permute(0, 1, 3, 4, 2)
        )

In [ ]:
class YOLOv3(nn.Module):
  def __init__(self, in_channels=3, NumClasses=20):
    super(YOLOv3, self).__init__()
    self.NumClasses = NumClasses
    self.in_channels = in_channels
    self.layers = self._create_conv_layers()

  def forward(self, x):
    outputs = []
    route_connections = []

    for layer in self.layers:
      if isinstance(layer, Prediction_Scale):
        outputs.append(layer(x))
        continue

      x = layer(x)

      if isinstance(layer, Residual_Block) and layer.num_repeats == 8:
        route_connections.append(x)

      elif isinstance(layer, nn.Upsample):
        x = torch.cat([x, route_connections[-1]], dim=1)
        route_connections.pop()

    return outputs

  def _create_conv_layers(self):
    layers = nn.ModuleList()
    in_channels = self.in_channels

    for module in config:
      if isinstance(module, tuple):
        out_channels, kernel_size, stride = module
        layers.append(CNN_Block(
            in_channels,
            out_channels,
            kernel_size=kernel_size,
            stride=stride,
            padding=1 if kernel_size == 3 else 0
        ))
        in_channels = out_channels

      elif isinstance(module, list):
        num_repeats = module[1]
        layers.append(Residual_Block(in_channels, num_repeats=num_repeats))

      elif isinstance(module, str):
        if module == "sp":
          layers += [
              Residual_Block(in_channels, use_residual=False, num_repeats=1),
              CNN_Block(in_channels, in_channels//2, kernel_size=1),
              Prediction_Scale(in_channels//2, NumClasses = self.NumClasses)
          ]
          in_channels = in_channels // 2

        elif module == "up":
          layers.append(nn.Upsample(scale_factor=2))
          in_channels = in_channels * 3

    return layers

In [ ]:
# Model hyperparameters
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
sp = [ImageSize // 32, ImageSize // 16, ImageSize // 8]

INPUT_DIM = tokenizer.vocab_size
OUTPUT_DIM = tokenizer.vocab_size
ImageSize = 512
WorkersNo = 4
HID_DIM = sp
ThresholdConf = 2
ThreshMap = 0.3
ThreshNms = 0.3

wandb.config.update({
    "RateOfLearning": 1e-3,
    "epochsno": 30,
    "SizeOfBatch": 64,
    "encoder_embedding_dim": ImageSize,
    "decoder_embedding_dim": WorkersNo,
    "hidden_dim": HID_DIM,
    "num_layers": ThresholdConf,
    "encoder_dropout": ThreshMap,
    "decoder_dropout": ThreshNms
})

optimizer = optim.Adam(model.parameters(), lr=wandb.config.learning_rate)
TRG_PAD_IDX = tokenizer.pad_token_id
criterion = nn.CrossEntropyLoss(ignore_index=TRG_PAD_IDX)

In [ ]:
NumClasses = 20
ImageSize = 416
model = YOLOv3(NumClasses=NumClasses)
x = torch.randn((2, 3, ImageSize, ImageSize))
out = model(x)
assert model(x)[0].shape == (2, 3, ImageSize//32, ImageSize//32, NumClasses + 5)
assert model(x)[1].shape == (2, 3, ImageSize//16, ImageSize//16, NumClasses + 5)
assert model(x)[2].shape == (2, 3, ImageSize//8, ImageSize//8, NumClasses + 5)
print("Success!")

In [ ]:
# Define utility functions
def WeidthHeight(boxa, boxb):
    intersection = torch.min(boxa[..., 0], boxb[..., 0]) * torch.min(
        boxa[..., 1], boxb[..., 1]
    )
    union = (
        boxa[..., 0] * boxa[..., 1] + boxb[..., 0] * boxb[..., 1] - intersection
    )
    return intersection / union

def non_max_suppression(boxx, iou_threshold, threshold, box_format="corners"):
    assert type(boxx) == list

    boxx = [box for box in boxx if box[1] > threshold]
    boxx = sorted(boxx, key=lambda x: x[1], reverse=True)
    boxx_after_nms = []

    while boxx:
        chosen_box = boxx.pop(0)

        boxx = [
            box
            for box in boxx
            if box[0] != chosen_box[0]
            or InterctionOverUnion(
                torch.tensor(chosen_box[2:]),
                torch.tensor(box[2:]),
                box_format=box_format,
            )
            < iou_threshold
        ]

        boxx_after_nms.append(chosen_box)

    return boxx_after_nms

def InterctionOverUnion(PredsBox, lableBox, box_format="midpoint"):
    if box_format == "midpoint":
        box1_a1 = PredsBox[..., 0:1] - PredsBox[..., 2:3] / 2
        box1_b1 = PredsBox[..., 1:2] - PredsBox[..., 3:4] / 2
        box1_a2 = PredsBox[..., 0:1] + PredsBox[..., 2:3] / 2
        box1_b2 = PredsBox[..., 1:2] + PredsBox[..., 3:4] / 2
        box2_a1 = lableBox[..., 0:1] - lableBox[..., 2:3] / 2
        box2_y1 = lableBox[..., 1:2] - lableBox[..., 3:4] / 2
        box2_a2 = lableBox[..., 0:1] + lableBox[..., 2:3] / 2
        box2_y2 = lableBox[..., 1:2] + lableBox[..., 3:4] / 2

    if box_format == "corners":
        box1_a1 = PredsBox[..., 0:1]
        box1_b1 = PredsBox[..., 1:2]
        box1_a2 = PredsBox[..., 2:3]
        box1_b2 = PredsBox[..., 3:4]
        box2_a1 = lableBox[..., 0:1]
        box2_y1 = lableBox[..., 1:2]
        box2_a2 = lableBox[..., 2:3]
        box2_y2 = lableBox[..., 3:4]

    x1 = torch.max(box1_a1, box2_a1)
    y1 = torch.max(box1_b1, box2_y1)
    x2 = torch.min(box1_a2, box2_a2)
    y2 = torch.min(box1_b2, box2_y2)
    intersection = torch.clamp(x2 - x1, min=0) * torch.clamp(y2 - y1, min=0)
    box1_area = (box1_a2 - box1_a1) * (box1_b2 - box1_b1)
    box2_area = (box2_a2 - box2_a1) * (box2_y2 - box2_y1)
    iou = intersection / (box1_area + box2_area - intersection)

    return iou

In [ ]:
#added own
# allows PIL to load images even if they are truncated or incomplete
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [ ]:
#added own imp
class YOLODataset(Dataset):
  def __init__(self, csv_file, ImgDir, LableDir, anchors,
               ImageSize=416, sp=[13,26,52], cp=20, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.ImgDir = ImgDir
    self.LableDir = LableDir
    self.transform = transform
    self.sp = sp

    self.anchors = torch.tensor(anchors[0] + anchors[1] + anchors[2]) # For all 3 scales
    self.num_anchors = self.anchors.shape[0]
    self.num_anchors_per_scale = self.num_anchors // 3

    self.cp = cp

    self.ignore_iou_thresh = 0.5

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, index):
    label_path = os.path.join(self.LableDir, self.annotations.iloc[index, 1])
    boxx = np.roll(np.loadtxt(fname=label_path, delimiter=" ", ndmin=2), 4, axis=1).tolist() # np.roll with shift 4 on axis 1: [class, x, y, w, h] --> [x, y, w, h, class]

    img_path = os.path.join(self.ImgDir, self.annotations.iloc[index, 0])
    image = Image.open(img_path)

    if self.transform:
      image = self.transform(image)

    targets = [torch.zeros((self.num_anchors // 3, sp, sp, 6)) for sp in self.sp] # 6 because objectness score, bounding box coordinates (x, y, w, h), class label

    for box in boxx:
      iou_anchors = WeidthHeight(torch.tensor(box[2:4]), self.anchors) # IOU from height and width
      anchor_indices = iou_anchors.argsort(descending=True, dim=0) # Sorting sucht that the first is the best anchor

      x, y, width, height, class_label = box
      has_anchor = [False, False, False] # Make sure there is an anchor for each of three scales for each bounding box

      for anchor_idx in anchor_indices:
        scale_idx = anchor_idx // self.num_anchors_per_scale # scale_idx is either 0,1,2: 0-->13x13, 1:-->26x26, 2:-->52x52
        anchor_on_scale = anchor_idx % self.num_anchors_per_scale # In each scale, choosing the anchor thats either 0,1,2

        sp = self.sp[scale_idx]
        i, j = int(sp*y), int(sp*x) # x=0.5, sp=13 --> int(6.5) = 6 | i=y cell, j=x cell
        anchor_taken = targets[scale_idx][anchor_on_scale, i, j, 0]

        if not anchor_taken and not has_anchor[scale_idx]:
          targets[scale_idx][anchor_on_scale, i, j, 0] = 1
          x_cell, y_cell = sp*x - j, sp*y - i # 6.5 - 6 = 0.5 such that they are between [0,1]
          width_cell, height_cell = (
              width*sp, # sp=13, width=0.5, 6.5
              height*sp
          )

          box_coordinates = torch.tensor([x_cell, y_cell, width_cell, height_cell])

          targets[scale_idx][anchor_on_scale, i, j, 1:5] = box_coordinates
          targets[scale_idx][anchor_on_scale, i, j, 5] = int(class_label)
          has_anchor[scale_idx] = True

        # Even if the same grid shares another anchor having iou>ignore_iou_thresh then,
        elif not anchor_taken and iou_anchors[anchor_idx] > self.ignore_iou_thresh:
          targets[scale_idx][anchor_on_scale, i, j, 0] = -1 # ignore this prediction

    return image, tuple(targets)

    # Define dataset
# class YOLODataset(Dataset):
#     def __init__(self, csv_file, transform=None):
#         self.data_frame = pd.read_csv(csv_file)
#         self.transform = transform

#     def __len__(self):
#         return len(self.data_frame)

#     def __getitem__(self, idx):
#         img_name = os.path.join(self.data_frame.iloc[idx, 0])
#         image = Image.open(img_name)
#         boxes = self.data_frame.iloc[idx, 1:]

#         if self.transform:
#             image = self.transform(image)

#         return image, boxes


In [ ]:
import torchvision.transforms as transforms
transform = transforms.Compose([transforms.Resize((416, 416)), transforms.ToTensor()])

In [ ]:
#added own imp
def get_loaders(train_csv_path, test_csv_path):

    train_dataset = YOLODataset(
        train_csv_path,
        transform=transform,
        sp=[ImageSize // 32, ImageSize // 16, ImageSize // 8],
        ImgDir=DirImage,
        LableDir=DirLable,
        anchors=ANCHORS,
    )
    test_dataset = YOLODataset(
        test_csv_path,
        transform=transform,
        sp=[ImageSize // 32, ImageSize // 16, ImageSize // 8],
        ImgDir=DirImage,
        LableDir=DirLable,
        anchors=ANCHORS,
    )
    train_loader = DataLoader(
        dataset=train_dataset,
        batch_size=SizeOfBatch,
        shuffle=True,
        drop_last=False,
    )
    test_loader = DataLoader(
        dataset=test_dataset,
        batch_size=SizeOfBatch,
        shuffle=False,
        drop_last=False,
    )

    return train_loader, test_loader

#     # Define data loaders
# def get_loaders(csv_file, batch_size, num_workers=0, shuffle=True):
#     transform = transforms.Compose([
#         transforms.ToTensor(),
#         transforms.Resize((416, 416)),
#     ])

#     dataset = YOLODataset(csv_file=csv_file, transform=transform)
#     return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=num_workers)

In [ ]:
def mean_average_precision(
    pred_boxes, true_boxes, iou_threshold=0.5, box_format="midpoint", NumClasses=4
):

    # list storing all AP for respective classes
    average_precisions = []

    # used for numerical stability later on
    epsilon = 1e-6

    for c in range(NumClasses):
        detections = []
        ground_truths = []

        for detection in pred_boxes:
            if detection[1] == c:
                detections.append(detection)

        for true_box in true_boxes:
            if true_box[1] == c:
                ground_truths.append(true_box)

        amount_boxx = Counter([gt[0] for gt in ground_truths])

        for key, val in amount_boxx.items():
            amount_boxx[key] = torch.zeros(val)

        # sort by box probabilities which is index 2
        detections.sort(key=lambda x: x[2], reverse=True)
        TP = torch.zeros((len(detections)))
        FP = torch.zeros((len(detections)))
        total_true_boxx = len(ground_truths)

        # If none exists for this class then we can safely skip
        if total_true_boxx == 0:
            continue

        for detection_idx, detection in enumerate(detections):
            ground_truth_img = [
                bbox for bbox in ground_truths if bbox[0] == detection[0]
            ]

            num_gts = len(ground_truth_img)
            best_iou = 0

            for idx, gt in enumerate(ground_truth_img):
                iou = InterctionOverUnion(
                    torch.tensor(detection[3:]),
                    torch.tensor(gt[3:]),
                    box_format=box_format,
                )

                if iou > best_iou:
                    best_iou = iou
                    best_gt_idx = idx

            if best_iou > iou_threshold:
                # only detect ground truth detection once
                if amount_boxx[detection[0]][best_gt_idx] == 0:
                    # true positive and add this bounding box to seen
                    TP[detection_idx] = 1
                    amount_boxx[detection[0]][best_gt_idx] = 1
                else:
                    FP[detection_idx] = 1

            # if IOU is lower then the detection is a false positive
            else:
                FP[detection_idx] = 1

        TP_cumsum = torch.cumsum(TP, dim=0)
        FP_cumsum = torch.cumsum(FP, dim=0)
        recalls = TP_cumsum / (total_true_boxx + epsilon)
        precisions = TP_cumsum / (TP_cumsum + FP_cumsum + epsilon)
        precisions = torch.cat((torch.tensor([1]), precisions))
        recalls = torch.cat((torch.tensor([0]), recalls))
        # torch.trapz for numerical integration
        average_precisions.append(torch.trapz(precisions, recalls))

    return sum(average_precisions) / len(average_precisions)

In [ ]:
#added own
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# WorkersNo = 4
# SizeOfBatch = 32
# ImageSize = 416
# ClassesNo = 20
# RateOfLearning = 1e-5
# epochsno = 150
# ThresholdConf = 0.8
# ThreshMap = 0.5
# ThreshNms = 0.45
# sp = [ImageSize // 32, ImageSize // 16, ImageSize // 8]

DirImage = "/kaggle/input/pascalvoc-yolo/images"
DirLable = "/kaggle/input/pascalvoc-yolo/labels"

ANCHORS = [
    [(0.28, 0.22), (0.38, 0.48), (0.9, 0.78)],
    [(0.07, 0.15), (0.15, 0.11), (0.14, 0.29)],
    [(0.02, 0.03), (0.04, 0.07), (0.08, 0.06)],
]


AllClacess = [
    "aeroplane",
    "bicycle",
    "bird",
    "boat",
    "bottle",
    "bus",
    "car",
    "cat",
    "chair",
    "cow",
    "diningtable",
    "dog",
    "horse",
    "motorbike",
    "person",
    "pottedplant",
    "sheep",
    "sofa",
    "train",
    "tvmonitor"
]

In [ ]:
def get_evaluation_boxx(
    loader,
    model,
    iou_threshold,
    anchors,
    threshold,
    box_format="midpoint"
):

    # make sure model is in eval before get boxx
    model.eval()
    train_idx = 0
    all_pred_boxes = []
    all_true_boxes = []
    for batch_idx, (x, labels) in enumerate(loader):
        x = x.float().to(DEVICE)

        with torch.no_grad():
            predictions = model(x)

        batch_size = x.shape[0]
        boxx = [[] for _ in range(batch_size)]
        for i in range(3):
            sp = predictions[i].shape[2] # grid cell size for each predictions
            anchor = torch.tensor([*anchors[i]]).to(DEVICE) * sp # anchor for each grid, prediction type
            boxes_scale_i = cells_to_boxx( # get boxx for each image in the batch
                predictions[i], anchor, sp=sp, is_preds=True
            )
            for idx, (box) in enumerate(boxes_scale_i): # for each image, append the bbox to corr. boxx[idx]
                boxx[idx] += box

        # we just want one bbox for each label, not one for each scale
        true_boxx = cells_to_boxx(
            labels[2], anchor, sp=sp, is_preds=False
        )

        for idx in range(batch_size):
            nms_boxes = non_max_suppression(
                boxx[idx],
                iou_threshold=iou_threshold,
                threshold=threshold,
                box_format=box_format,
            )

            for nms_box in nms_boxes:
                all_pred_boxes.append([train_idx] + nms_box)

            for box in true_boxx[idx]:
                if box[1] > threshold:
                    all_true_boxes.append([train_idx] + box)

            train_idx += 1

    model.train()
    return all_pred_boxes, all_true_boxes

In [ ]:
#added own imp
def cells_to_boxx(predictions, anchors, sp, is_preds=True):

    SizeOfBatch = predictions.shape[0]
    num_anchors = len(anchors)
    box_predictions = predictions[..., 1:5]
    if is_preds:
        anchors = anchors.reshape(1, len(anchors), 1, 1, 2)
        box_predictions[..., 0:2] = torch.sigmoid(box_predictions[..., 0:2])
        box_predictions[..., 2:] = torch.exp(box_predictions[..., 2:]) * anchors
        scores = torch.sigmoid(predictions[..., 0:1])
        best_class = torch.argmax(predictions[..., 5:], dim=-1).unsqueeze(-1)
    else:
        scores = predictions[..., 0:1]
        best_class = predictions[..., 5:6]

    cell_indices = (
        torch.arange(sp)
        .repeat(predictions.shape[0], 3, sp, 1)
        .unsqueeze(-1)
        .to(predictions.DEVICE)
    )
    x = 1 / sp * (box_predictions[..., 0:1] + cell_indices)
    y = 1 / sp * (box_predictions[..., 1:2] + cell_indices.permute(0, 1, 3, 2, 4))
    w_h = 1 / sp * box_predictions[..., 2:4]
    converted_boxx = torch.cat((best_class, scores, x, y, w_h), dim=-1).reshape(SizeOfBatch, num_anchors * sp * sp, 6)
    return converted_boxx.tolist()

# Define bounding box conversion
# def cells_to_boxx(prediction, anchors, img_size=416):
#     grid_size = prediction.shape[2]
#     box_x, box_y = torch.meshgrid([torch.arange(grid_size)] * 2, indexing='ij')
#     box_x = box_x.to(prediction.DEVICE)
#     box_y = box_y.to(prediction.DEVICE)

#     prediction = prediction.reshape(-1, grid_size, grid_size, 3, 5 + 20)
#     prediction[..., :2] = torch.sigmoid(prediction[..., :2]) + torch.cat([box_x.unsqueeze(-1), box_y.unsqueeze(-1)], dim=-1).float()
#     prediction[..., 2:4] = torch.exp(prediction[..., 2:4]) * torch.tensor(anchors).to(prediction.DEVICE).unsqueeze(0).unsqueeze(0)
#     prediction[..., :4] *= img_size

#     return prediction

In [ ]:
#added own imp
class YoloLoss(nn.Module):
  def __init__(self):
    super(YoloLoss, self).__init__()
    self.mse = nn.MSELoss() # For bounding box loss
    self.bce = nn.BCEWithLogitsLoss() # For multi-label prediction: Binary cross entropy
    self.entropy = nn.CrossEntropyLoss() # For classification
    self.sigmoid = nn.Sigmoid()

    # Constants for significance of obj, or no obj.
    self.lambda_class = 1
    self.lambda_noobj = 10
    self.lambda_obj = 1
    self.lambda_box = 10

  def forward(self, predictions, target, anchors):
    obj = target[..., 0] == 1
    noobj = target[..., 0] == 0

    no_object_loss = self.bce(
        (predictions[..., 0:1][noobj]), (target[..., 0:1][noobj])
    )

    anchors = anchors.reshape(1,3,1,1,2) # Anchors initial shape 3x2 --> 3 anchor boxes each of certain hxw (2)

    # box_preds = [..., sigmoid(x), sigmoid(y), [p_w * exp(t_w)], [p_h * exp(t_h)], ...]
    box_preds = torch.cat([self.sigmoid(predictions[..., 1:3]), torch.exp(predictions[..., 3:5]) * anchors], dim=-1)

    # iou between predicted box and target box
    ious = InterctionOverUnion(box_preds[obj], target[..., 1:5][obj]).detach()

    object_loss = self.bce(
        (predictions[..., 0:1][obj]), (ious * target[..., 0:1][obj]) # target * iou because only intersected part object loss calc
    )

    predictions[..., 1:3] = self.sigmoid(predictions[..., 1:3]) # x, y to be between [0,1]
    target[..., 3:5] = torch.log(
        (1e-6 + target[..., 3:5] / anchors)
    ) # Exponential of hxw (taking log because opp. of exp)

    box_loss = self.mse(predictions[..., 1:5][obj], target[..., 1:5][obj])

    class_loss = self.entropy(
        (predictions[..., 5:][obj]), (target[..., 5][obj].long())
    )

    return(
        self.lambda_box * box_loss
        + self.lambda_obj * object_loss
        + self.lambda_noobj * no_object_loss
        + self.lambda_class * class_loss
    )

# Define loss function for YOLOv3
# class YoloLoss(nn.Module):
#     def __init__(self, anchors, num_classes):
#         super(YoloLoss, self).__init__()
#         self.anchors = anchors
#         self.num_classes = num_classes

#     def forward(self, predictions, targets):
#         # Implement loss calculation here
#         return loss


In [ ]:
#added own imp
def plot_image(image, boxes):
    """Plots predicted bounding boxes on the image"""
    cmap = plt.get_cmap("tab20b")
    class_labels = AllClacess
    colors = [cmap(i) for i in np.linspace(0, 1, len(class_labels))]
    im = np.array(image)
    height, width, _ = im.shape

    # Create figure and axes
    fig, ax = plt.subplots(1)
    # Display the image
    ax.imshow(im)

    # Create a Rectangle patch
    for box in boxes:
        assert len(box) == 6, "box should contain class pred, confidence, x, y, width, height"
        class_pred = box[0]
        box = box[2:]
        UpperLeft_x = box[0] - box[2] / 2
        UpperLeft_y = box[1] - box[3] / 2
        rect = patches.Rectangle(
            (UpperLeft_x * width, UpperLeft_y * height),
            box[2] * width,
            box[3] * height,
            linewidth=2,
            edgecolor=colors[int(class_pred)],
            facecolor="none",
        )
        # Add the patch to the Axes
        ax.add_patch(rect)
        plt.text(
            UpperLeft_x * width,
            UpperLeft_y * height,
            s=class_labels[int(class_pred)],
            color="white",
            verticalalignment="top",
            bbox={"color": colors[int(class_pred)], "pad": 0},
        )

    plt.show()

# # Define plotting function
#     def plot_image(img, boxes):
#     fig, ax = plt.subplots(1, figsize=(12, 9))
#     ax.imshow(img)

#     for box in boxes:
#         x, y, w, h, conf, cls = box
#         rect = patches.Rectangle((x, y), w, h, linewidth=1, edgecolor='r', facecolor='none')
#         ax.add_patch(rect)
#         plt.text(x, y, f'{cls} {conf:.2f}', color='red', fontsize=12)

#     plt.show()

In [ ]:
import torch.optim as optim

from tqdm import tqdm
import time

history_loss = []

for epoch in tqdm(range(epochsno), desc="Epochs"):
  model.train()

  losses = []

  start_time = time.time()

  for batch_idx, (x,y) in enumerate(train_loader):
    x = x.to(DEVICE)
    y0, y1, y2 = (y[0].to(DEVICE),
                  y[1].to(DEVICE),
                  y[2].to(DEVICE))

    with torch.cuda.amp.autocast():
      out = model(x)
      loss = (
          loss_fn(out[0], y0, scaled_anchors[0])
          + loss_fn(out[1], y1, scaled_anchors[1])
          + loss_fn(out[2], y2, scaled_anchors[2])
      )

    losses.append(loss.item())

    optimizer.zero_grad()
    scaler.scale(loss).backward()
    scaler.step(optimizer)
    scaler.update()

  end_time = time.time()
  epoch_duration = end_time - start_time

  history_loss.append(sum(losses)/len(losses))

  if (epoch+1) % 10 == 0:
    tqdm.write(f"Epoch {epoch+1} completed in {epoch_duration:.2f} seconds")

    print(f"Epoch [{epoch+1}/{epochsno}], "
          f"Loss: {sum(losses)/len(losses):.4f}")

    torch.save(model.state_dict(), f'/kaggle/working/Yolov3_epoch{epoch+1}.pth')


Epochs:   0%|          | 0/150 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/amp/autocast_mode.py:250: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
Epochs:   6%|▌         | 9/150 [09:28<2:11:03, 55.77s/it]

Epoch 10 completed in 55.52 seconds
Epoch [10/150], Loss: 26.0588


Epochs:  13%|█▎        | 19/150 [18:45<1:59:51, 54.89s/it]

Epoch 20 completed in 53.61 seconds
Epoch [20/150], Loss: 16.6168


Epochs:  19%|█▉        | 29/150 [28:07<1:54:46, 56.91s/it]

...

Epochs:  53%|█████▎    | 79/150 [1:18:04<1:07:11, 56.78s/it]

Epoch 80 completed in 54.13 seconds
Epoch [80/150], Loss: 6.0814


Epochs:  59%|█████▉    | 89/150 [1:27:21<55:49, 54.91s/it]

Epoch 90 completed in 54.73 seconds
Epoch [90/150], Loss: 6.0831


Epochs:  61%|██████    | 91/150 [1:28:18<54:30, 55.43s/it]

In [ ]:
# Instantiate the model
model = YOLOv3(NumClasses=NumClasses).to(DEVICE)

# Compile the model
optimizer = torch.optim.Adam(
    model.parameters(), lr=RateOfLearning
)
loss_fn = YoloLoss()

# Scaler
scaler = torch.cuda.amp.GradScaler()

# Train-Test Loader
train_loader, test_loader = get_loaders(
    train_csv_path='/kaggle/input/pascalvoc-yolo/8examples.csv', test_csv_path='/kaggle/input/pascalvoc-yolo/8examples.csv'
)

# Anchors
scaled_anchors = (
    torch.tensor(ANCHORS) * torch.tensor([13,26,52]).unsqueeze(1).unsqueeze(1).repeat(1,3,2)
).to(DEVICE)

In [ ]:
# # Example usage
# if __name__ == "__main__":

#     for epoch in range(10):
#         model.train()
#         running_loss = 0.0

#         for images, targets in tqdm(train_loader):
#           if images is None or targets is None:
#             continue  # Skip the iteration if the image or target is None

#             optimizer.zero_grad()
#             outputs = model(images)
#             loss = criterion(outputs, targets)
#             loss.backward()
#             optimizer.step()
#             running_loss += loss.item()

#         avg_loss = running_loss / len(train_loader)
#         print(f'Epoch {epoch+1}, Loss: {avg_loss}')

#         # Log metrics to wandb
#         wandb.log({
#             "epoch": epoch + 1,
#             "loss": avg_loss
#         })

#         # Optionally log model
#         wandb.watch(model, log="all", log_freq=10)

#     # Save the model
#     torch.save(model.state_dict(), "yolov3_model.pth")
#     wandb.save("yolov3_model.pth")


In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

if not os.path.exists('checkpoints'):
    os.makedirs('checkpoints')

N_EPOCHS = wandb.config.epochs
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time()
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    wandb.log({"train_loss": train_loss, "train_accuracy": train_acc,
               "valid_loss": valid_loss, "valid_accuracy": valid_acc, "valid_bleu": bleu,
               "epoch": epoch + 1, "epoch_time_mins": epoch_mins, "epoch_time_secs": epoch_secs})

wandb.finish()


In [ ]:
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Track loss
        running_loss += loss.item()

        # Convert outputs to predictions
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

    # Calculate F1 score for the epoch
    epoch_f1 = f1_score(all_labels, all_preds, average='weighted')

    # Log metrics to wandb
    wandb.log({"epoch": epoch, "loss": running_loss / len(data_loader), "f1_score": epoch_f1})


In [ ]:
    for inputs, labels in tqdm(train_loader):
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

    epoch_loss = running_loss / len(train_loader)

    # Calculate metrics
    precision = precision_score(all_labels, all_preds, average='macro')
    recall = recall_score(all_labels, all_preds, average='macro')
    f1 = f1_score(all_labels, all_preds, average='macro')

    # Log metrics to wandb
    wandb.log({
        "epoch": epoch,
        "loss": epoch_loss,
        "precision": precision,
        "recall": recall,
        "f1_score": f1
    })

print("Training complete.")

# Inference on a single image
def infer_image(image_path):
    model.eval()
    image = Image.open(image_path).convert('L')  # Convert to grayscale
    image = transform(image).unsqueeze(0)  # Add batch dimension
    with torch.no_grad():
        output = model(image)
        _, pred = torch.max(output, 1)
        print(f'Predicted class: {pred.item()}')

# Example usage for inference
infer_image('path_to_your_image.png')